In [2]:
import requests

url = "https://api.github.com/repos/huggingface/datasets/issues?page=1&per_page=1"
response = requests.get(url)

In [3]:
response.status_code

200

In [4]:
response.json()

[{'url': 'https://api.github.com/repos/huggingface/datasets/issues/7858',
  'repository_url': 'https://api.github.com/repos/huggingface/datasets',
  'labels_url': 'https://api.github.com/repos/huggingface/datasets/issues/7858/labels{/name}',
  'comments_url': 'https://api.github.com/repos/huggingface/datasets/issues/7858/comments',
  'events_url': 'https://api.github.com/repos/huggingface/datasets/issues/7858/events',
  'html_url': 'https://github.com/huggingface/datasets/pull/7858',
  'id': 3605471548,
  'node_id': 'PR_kwDODunzps6yZq4r',
  'number': 7858,
  'title': 'Support downloading specific splits in `load_dataset`',
  'user': {'login': 'CloseChoice',
   'id': 31857876,
   'node_id': 'MDQ6VXNlcjMxODU3ODc2',
   'avatar_url': 'https://avatars.githubusercontent.com/u/31857876?v=4',
   'gravatar_id': '',
   'url': 'https://api.github.com/users/CloseChoice',
   'html_url': 'https://github.com/CloseChoice',
   'followers_url': 'https://api.github.com/users/CloseChoice/followers',
   'f

Create a .env file in the directory and out a single entry GITHUB_TOKEN=XXX in the file.

In [5]:
from dotenv import load_dotenv
import os

In [6]:
load_dotenv()

True

In [7]:
headers = {"Authorization": f"token {os.environ['GITHUB_TOKEN']}"}

In [8]:
import time
import math
from pathlib import Path
import pandas as pd
from tqdm.notebook import tqdm


def fetch_issues(
    owner="huggingface",
    repo="datasets",
    num_issues=10_000,
    rate_limit=5_000,
    issues_path=Path("."),
):
    if not issues_path.is_dir():
        issues_path.mkdir(exist_ok=True)

    batch = []
    all_issues = []
    per_page = 100  # Number of issues to return per page
    num_pages = math.ceil(num_issues / per_page)
    base_url = "https://api.github.com/repos"

    for page in tqdm(range(num_pages)):
        # Query with state=all to get both open and closed issues
        query = f"issues?page={page}&per_page={per_page}&state=all"
        issues = requests.get(f"{base_url}/{owner}/{repo}/{query}", headers=headers)
        batch.extend(issues.json())

        if len(batch) > rate_limit and len(all_issues) < num_issues:
            all_issues.extend(batch)
            batch = []  # Flush batch for next time period
            print(f"Reached GitHub rate limit. Sleeping for one hour ...")
            time.sleep(60 * 60 + 1)

    all_issues.extend(batch)
    df = pd.DataFrame.from_records(all_issues)
    df.to_json(f"{issues_path}/{repo}-issues.jsonl", orient="records", lines=True)
    print(
        f"Downloaded all the issues for {repo}! Dataset stored at {issues_path}/{repo}-issues.jsonl"
    )

In [9]:
fetch_issues()

  0%|          | 0/100 [00:00<?, ?it/s]

Downloaded all the issues for datasets! Dataset stored at ./datasets-issues.jsonl
